# Backtesting the "Cramer Effect"

**The cramer-effect/cramer-bounce**: After the show *Mad Money* the recommended stocks are bought by viewers almost immediately (afterhours trading) or on the next day at market open, increasing the price for a short period of time.

**Facts:**
- Mad money is at 6:00 PM ET
    - (This is 10:00 PM UTC+0) 
- NYSE is open from 9:30 AM to 4:00 PM ET
    - (This is 1:30 PM to 8:00 PM UTC+0)

**Because of the above:**
- People buy the mentioned stocks
    - After the show (7:00 PM ET) (+/- 30mins)
    - Next trading day at Open (+/- 30mins)
- The increase is for a short period of time, but there are no exact figures, we can test on multiple hold periods
    - Next day Close
    - 1, 2, 3, ... days at Close
- (If the show was on Friday night, we won't care about it and won't buy or sell) --> TODO: Should we use the next available business date? e.g. monday?

Refs:
- https://www.investopedia.com/terms/c/cramerbounce.asp
- https://www.kiplinger.com/article/investing/t031-c023-s001-the-cramer-effect.html
- https://www.davemanuel.com/investor-dictionary/cramer-effect/

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import Counter
from datetime import datetime, timedelta
from typing import List
import concurrent.futures
import warnings

import numpy as np
import pandas as pd
import backtesting
from tqdm import tqdm
import yfinance

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import mad_money_backtesting as mmb

C:\Users\vecse\anaconda3\lib\site-packages\backtesting\_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
warnings.simplefilter("ignore")

# Constants

In [4]:
BACKTEST_FROM = "2020-01-01"
CASH_PER_STOCK = 1000 # if a stocks price is bigger than this, we will skip it
COMMISSION_RATE = 0.02

# Read the csv

In [5]:
df = pd.read_csv("mad_money.csv", parse_dates=["date"])

In [6]:
df.head()

,Unnamed: 0,name,month_and_day,segment,call,current_price,date,symbol
0,0,CrowdStrike (CRWD),01/03,I,buy,50.75,2020-01-03,CRWD
1,1,Lam Research (LRCX),01/03,D,buy,294.69,2020-01-03,LRCX
2,2,Microsoft (MSFT),01/03,D,buy,158.62,2020-01-03,MSFT
3,3,Nike (NKE),01/03,D,buy,101.92,2020-01-03,NKE
4,4,Procter & Gamble (PG),01/03,D,buy,122.58,2020-01-03,PG


# Filter the data

## Call type filter

We only care about the "buy" mentions, but this can be extended to the "positive" mentions as well

In [7]:
df = df[df["call"] == "buy"]

# Minimal Stats

In [8]:
print(f"Number of unique symbols in selected range: {len(df['symbol'].unique())}")

Number of unique symbols in selected range: 733


In [9]:
c = Counter(df['symbol'].values).most_common()

In [10]:
print(f"Top 5 Symbol mentions from Cramer: {c[:5]}")

Top 5 Symbol mentions from Cramer: [('AAPL', 76), ('TSLA', 63), ('ZM', 45), ('COST', 41), ('HD', 41)]


# Backtest all mentioned stocks

## Filter based on the date

In [11]:
if BACKTEST_FROM is None:
    BACKTEST_FROM = df["date"].values[0]

In [12]:
df = df[df["date"] >= BACKTEST_FROM]

In [13]:
print(f"Backtesting starts from {df.date.values[0]} and lasts until {df.date.values[-1]}")

Backtesting starts from 2020-01-03T00:00:00.000000000 and lasts until 2021-05-21T00:00:00.000000000


## Filter based on price

In [14]:
df = df[df["current_price"] <= CASH_PER_STOCK]

## Filter Fridays - TODO: do we always want to filter them?

In [15]:
df = df[df["date"].dt.day_name() != "Friday"]

## Define backtesting for a single stock

TODO: writeup on how the backtesting goes and how buy and sell dates calculated

In [16]:
def backtest_single_stock(symbol, mad_money_df, cash, commission, buy_date_fn, sell_date_fn):
    # Get the dates when the Stock was recommended
    recommendation_dates = mad_money_df[mad_money_df["symbol"] == symbol]["date"]
    recommendation_dates = [mmb.pd_date_to_datetime(x) for x in recommendation_dates]
    
    # The difference between the first and last mention
    # nb_days = len(pd.bdate_range(recommendation_dates[0], recommendation_dates[-1]))
    # if nb_days > 730:
    #     raise ValueError(f"We cannot download data for this range. It's more than 730 days. It's {nb_days}")
    nb_days = 300
    
    # Calculate when to buy and when to sell a stock
    buy_dates = [buy_date_fn(x) for x in recommendation_dates]
    sell_dates = [sell_date_fn(x) for x in buy_dates]
    
    # Download the stock data - with pre and post data
    # TODO: number of days should be a parameter
    stock_df = yfinance.Ticker(symbol).history(period=f"{nb_days}d", interval="1h", prepost=True)
    if len(stock_df) < 1:
        raise ValueError(f"There is not data in the dataframe for: {symbol}")
    stock_df["Date"] = stock_df.index
    stock_df.dropna(inplace=True)
    
    # Run the backtesting with our strategy
    bt = backtesting.Backtest(stock_df, mmb.MadMoneyStrategy, cash=cash, commission=commission, trade_on_close=True)
    results = bt.run(buy_dates=buy_dates, sell_dates=sell_dates, buy_size=0.999)
    
    return bt, results

## Buy and Sell date calculation

- Buy dates are calculated from the "recommendation dates" (date when Cramer mentioned the stock)
- Sell dates are calculated from the previously calculated Buy dates

Pay attention to the time adjustment. From Marc 28. to Oct 31

This is done under the hood, you'll need to define 2 functions just like this:

```python
def buy_at_next_day_open(date_of_recommendation):
    # We want to buy the next market open
    return mmb.pd_date_to_datetime(date_of_recommendation, 9, 30) + timedelta(days=1)


def sell_at_next_day_close(date_of_last_buy):
    # We want to sell at the end of the trading day (when we bought it)
    return date_of_last_buy.replace(hour=16, minute=0)
```

In [17]:
### Functions to Buy at next market day's Open and Sell at the end of the day ###

# def buy_at_next_day_open(date_of_recommendation):
#     return mmb.pd_date_to_datetime(date_of_recommendation, 9, 30) + timedelta(days=1)
# 
# 
# def sell_at_next_day_close(date_of_last_buy):
#     return date_of_last_buy.replace(hour=16, minute=0)

### Functions to Buy at the end of the show (7:00 PM ET) and sell next day Close or Open ###

def buy_at_the_end_of_the_show(date_of_recommendation):
    return mmb.pd_date_to_datetime(date_of_recommendation, 19, 0)


def sell_at_next_day_close(date_of_last_buy):
    return date_of_last_buy.replace(hour=16, minute=0) + timedelta(days=1)


def sell_at_next_day_open(date_of_last_buy):
    return date_of_last_buy.replace(hour=9, minute=30) + timedelta(days=1)

In [18]:
buy_date_calc_fn = buy_at_the_end_of_the_show
sell_day_calc_fn = sell_at_next_day_close

In [19]:
stocks_to_backtest = df["symbol"].unique()[:10]
pbar = tqdm(total=len(stocks_to_backtest))

bt_results = {}

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {}
    
    for symbol in stocks_to_backtest:
        f = executor.submit(backtest_single_stock,
                            symbol=symbol,
                            mad_money_df=df,
                            cash=CASH_PER_STOCK,
                            commission=COMMISSION_RATE,
                            buy_date_fn=buy_date_calc_fn,
                            sell_date_fn=sell_day_calc_fn)
        futures[f] = symbol
        
    for f in concurrent.futures.as_completed(futures):
        symbol = futures[f]
        
        try:
            bt_obj, bt_result = f.result()
            bt_results[symbol] = {"obj":bt_obj, "result":bt_result}
        except Exception as e:
            print(f"There was a problem with {symbol} - {e}")
            
        pbar.update(1)
            
pbar.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.99it/s]


In [20]:
results_df = mmb.summarize_backtesting_results(results=[x["result"] for x in bt_results.values()],
                                               symbols=list(bt_results.keys()),
                                               include_parameters=False,
                                               sort_by="Return [%]")

In [21]:
results_df.style.background_gradient(cmap="magma_r")

,Return [%],Equity Final [$],Equity Peak [$],Buy & Hold Return [%],Start,End
Strategy,,,,,,
MadMoneyStrategy (PXD),0.256680,1002.566800,1054.130826,150.516879,2020-03-16 08:00:00-04:00,2021-05-21 19:00:00-04:00
MadMoneyStrategy (GLD),0.000000,1000.000000,1000.000000,22.132908,2020-03-16 04:00:00-04:00,2021-05-21 19:00:00-04:00
MadMoneyStrategy (KSS),0.000000,1000.000000,1000.000000,145.252158,2020-03-16 04:00:00-04:00,2021-05-21 19:00:00-04:00
MadMoneyStrategy (SLB),0.000000,1000.000000,1000.000000,113.288591,2020-03-16 04:00:00-04:00,2021-05-21 19:00:00-04:00
MadMoneyStrategy (AMRN),0.000000,1000.000000,1000.000000,-58.090909,2020-03-16 04:00:00-04:00,2021-05-21 18:00:00-04:00
MadMoneyStrategy (PLNT),-0.750520,992.494800,1015.374780,59.316327,2020-03-16 07:00:00-04:00,2021-05-21 17:00:00-04:00
MadMoneyStrategy (MKTX),-2.589340,974.106600,1000.000000,46.597010,2020-03-16 09:00:00-04:00,2021-05-21 16:00:00-04:00
MadMoneyStrategy (LHX),-3.270000,967.300000,1000.000000,35.253243,2020-03-16 09:30:00-04:00,2021-05-21 16:00:00-04:00
MadMoneyStrategy (ED),-15.050000,849.500000,1000.000000,-8.459845,2020-03-16 07:00:00-04:00,2021-05-21 19:00:00-04:00
